## 신생아

In [24]:
import pandas as pd
import numpy as np

from statsmodels.formula.api import ols
import statsmodels.api as sm

from sklearn.metrics import classification_report
from sklearn.metrics import auc
from sklearn.metrics import roc_curve

#### [문제 5] 데이터 파일(birth_risk.csv, birth_risk_br.csv)을 통해 신생아와 산모의 데이터를 구축한다. 데이터의 변수명과 변수의 타입은 위의 변수 설명을 참고해 변경하시오. immature는 "37주 미만"을 참조 레벨로, plural은 "한명"을 참조 레벨로 정의한다. 임신 전 산모의 체중 변수에서 이상치들을 제거 후 데이터의 개수를 표시하시오. 이상치는 Q1(25%)을 기준으로 IQR(Q3 - Q1)의 1.5배 보다 작은(미만) 값들과, Q3(75%)을 기준으로 IQR의 1.5배 보다 큰(초과) 값들로 정의해 계산하시오. (10점)
- Brightics의 경우 범주형 변수로부터 Dummy 변수를 생성할 때는 변수값의 고유한 개수(k)에서 하나를 뺀 개수만큼(k-1)의 Dummy 변수를 만드시오. 예를 들어 원래 변수가 성별(남, 여)이라면 남성 여부 또는 여성 여부 둘 중에 하나만 만든다. (범주의 개수가 2개인 경우 Dummy 변수 1개 생성)
- Brightics의 경우 참조 레벨은 고려하지 않고 생성하시오.

(답안예시) 123 


In [2]:
df = pd.read_csv("birth.csv")
df.head(2)

,weight,visits,smoke,immature,plural,amniotic_fluid,parturition,breech,glycosuria,chypertension,phypertension,gestosis,xgrp,at_risk
0,68.934240,12,False,37주이상,한명,False,False,False,False,False,False,False,8,False
1,88.435374,9,False,37주이상,한명,False,False,False,False,False,False,False,2,False


In [3]:
df.apply(lambda x: set(x), axis = 0)

weight            {102.947845804989, 103.40136054421801, 103.854...
visits            {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
smoke                                                 {False, True}
immature                                             {37주이상, 37주미만}
plural                                            {쌍둥이, 한명, 세쌍둥이이상}
amniotic_fluid                                        {False, True}
parturition                                           {False, True}
breech                                                {False, True}
glycosuria                                            {False, True}
chypertension                                         {False, True}
phypertension                                         {False, True}
gestosis                                              {False, True}
xgrp                             {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
at_risk                                               {False, True}
dtype: object

In [4]:
df["immature"] = (df["immature"] == "37주이상") + 0
df.head(2)

,weight,visits,smoke,immature,plural,amniotic_fluid,parturition,breech,glycosuria,chypertension,phypertension,gestosis,xgrp,at_risk
0,68.934240,12,False,1,한명,False,False,False,False,False,False,False,8,False
1,88.435374,9,False,1,한명,False,False,False,False,False,False,False,2,False


In [5]:
for n in np.r_[2, 5:14]:
    df.iloc[:, n] = df.iloc[:, n] + 0
df.head(2)

,weight,visits,smoke,immature,plural,amniotic_fluid,parturition,breech,glycosuria,chypertension,phypertension,gestosis,xgrp,at_risk
0,68.934240,12,0,1,한명,0,0,0,0,0,0,0,8,0
1,88.435374,9,0,1,한명,0,0,0,0,0,0,0,2,0


In [6]:
df = pd.get_dummies(df)
df.head(2)

,weight,visits,smoke,immature,amniotic_fluid,parturition,breech,glycosuria,chypertension,phypertension,gestosis,xgrp,at_risk,plural_세쌍둥이이상,plural_쌍둥이,plural_한명
0,68.934240,12,0,1,0,0,0,0,0,0,0,8,0,0,0,1
1,88.435374,9,0,1,0,0,0,0,0,0,0,2,0,0,0,1


In [7]:
df = df.iloc[:, :-1]
df.head(2)

,weight,visits,smoke,immature,amniotic_fluid,parturition,breech,glycosuria,chypertension,phypertension,gestosis,xgrp,at_risk,plural_세쌍둥이이상,plural_쌍둥이
0,68.934240,12,0,1,0,0,0,0,0,0,0,8,0,0,0
1,88.435374,9,0,1,0,0,0,0,0,0,0,2,0,0,0


In [8]:
weight_q1 = df["weight"].quantile(0.25)
weight_q3 = df["weight"].quantile(0.75)
print(weight_q1, weight_q3)

56.6893424036281 76.1904761904762


In [9]:
IQR = weight_q3 - weight_q1
print(IQR)

19.5011337868481


In [10]:
df_sub = df.loc[(df["weight"] >= (weight_q1 - 1.5 * IQR)) & (df["weight"] <= (weight_q3 + 1.5 * IQR)),]
print(len(df))
print(len(df_sub))

25000
24636


#### [문제 6] [문제 5]의 결과 데이터 프레임에서 변수 xgrp의 값이 5 이하인 데이터로 훈련 데이터 셋을, 6 이상의 데이터로 테스트 데이터 셋을 만들고, xgrp 변수를 제외한 나머지 변수를 이용해 신생아의 위험 여부를 판단하는 로지스틱 회귀(Logistic Regression) 모델을 구축하시오. 로지스틱 회귀 모델의 계수는 입력변수와 출력변수 간의 관계를 설명한다. 
생성된 모델의 모든 조건이 동일하다는 전제 하에 정상 분만의 신생아가 미숙아보다 위험에 빠질 오즈(Odds)는 몇 배 줄어드는지, 계산하여 답하시오. (5점)
- 로지스틱 회귀모델은 훈련 데이터 셋으로 구축하시오.
- 소수점 둘째 자리에서 반올림하고, 소수점 첫째 자리까지 표시하시오.

(답안예시) 1.2

In [11]:
columns = df_sub.columns[:11].append(df_sub.columns[12:])
columns

Index(['weight', 'visits', 'smoke', 'immature', 'amniotic_fluid',
       'parturition', 'breech', 'glycosuria', 'chypertension', 'phypertension',
       'gestosis', 'at_risk', 'plural_세쌍둥이이상', 'plural_쌍둥이'],
      dtype='object')

In [12]:
train = df_sub.loc[df_sub["xgrp"] <= 5, columns]
test = df_sub.loc[df_sub["xgrp"] >= 6, columns]
train.head(2)

,weight,visits,smoke,immature,amniotic_fluid,parturition,breech,glycosuria,chypertension,phypertension,gestosis,at_risk,plural_세쌍둥이이상,plural_쌍둥이
1,88.435374,9,0,1,0,0,0,0,0,0,0,0,0,0
2,78.911565,9,0,1,0,0,0,0,0,0,0,0,0,0


In [26]:
model = sm.Logit(train["at_risk"], train.drop("at_risk", axis = 1))
model_fit = model.fit()
model_fit.summary()

Optimization terminated successfully.
         Current function value: 0.084424
         Iterations 9


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                at_risk   No. Observations:                13321
Model:                          Logit   Df Residuals:                    13308
Method:                           MLE   Df Model:                           12
Date:                Fri, 23 Aug 2019   Pseudo R-squ.:                 0.04509
Time:                        10:36:41   Log-Likelihood:                -1124.6
converged:                       True   LL-Null:                       -1177.7
Covariance Type:            nonrobust   LLR p-value:                 3.369e-17
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
weight            -0.0251      0.003     -9.840      0.000      -0.030      -0.020
visits            -0.1139      0.015     -7.573      0.000      -0.143      -0.084
smoke              0.1410      0.202      0.699      0.485      -0.254       0.536
immature          -1.6888      0.140    -12.028      0.000      -1.964      -1.414
amniotic_fluid     0.7303      0.255      2.858      0.004       0.230       1.231
parturition       -0.2851      0.429     -0.664      0.507      -1.126       0.556
breech             0.3995      0.201      1.983      0.047       0.005       0.794
glycosuria        -0.2458      0.320     -0.769      0.442      -0.872       0.381
chypertension      0.2798      0.530      0.528      0.597      -0.758       1.318
phypertension      0.1307      0.282      0.463      0.643      -0.422       0.684
gestosis           0.3325      1.063      0.313      0.754      -1.752       2.417
plural_세쌍둥이이상      1.2873      0.586      2.195      0.028       0.138       2.437
plural_쌍둥이         0.5100      0.250      2.038      0.042       0.019       1.001
==================================================================================
"""

In [17]:
round(np.exp(-1.6888), 1)

0.2

#### [문제 7] [문제 6]의 결과 모델을 이용하여 산모가 임신 중 병원을 방문할 때마다 신생아의 위험에 빠질 오즈(Odds)는 몇 배 줄어든다고 할 수 있는지 계산하여 답하시오. (5점)
- 소수점 둘째 자리에서 반올림하고, 소수점 첫째 자리까지 표시하시오

(답안예시) 1.5

In [18]:
round(np.exp(-0.1139), 1)

0.9

#### [문제 8] [문제 6]에서 생성한 로지스틱 회귀 모델을 이용해 다음의 입력 데이터에 대하여 예측 결과를 백분율(%)과 "위험" 또는 "정상"으로 표시하십시오. 이 때 분류를 위한 경계값으로 0.02를 사용합니다. (백분율은 소수점 둘째 자리에서 반올림하고, 소수점 첫째 자리까지 표시하고, XX.X%(정상) 또는 XX.X%(위험)과 같이 기술하십시오.) (10점)

입력 데이터: weight: 59, visits: 8, smoke: FALSE, immature: "37주미만", plural: "쌍둥이", amniotic_fluid: FALSE, parturition: FALSE, breech: TRUE, glycosuria: FALSE, chypertension: FALSE, phypertension: FALSE, gestosis: FALSE


In [19]:
test_one = pd.DataFrame(columns = train.columns,
                        data = [[59, 8, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]])
test_one

,weight,visits,smoke,immature,amniotic_fluid,parturition,breech,glycosuria,chypertension,phypertension,gestosis,at_risk,plural_세쌍둥이이상,plural_쌍둥이
0,59,8,0,0,0,0,1,0,0,0,0,0,0,1


In [20]:
model_fit.predict(test_one.drop("at_risk", axis = 1))

0    0.184955
dtype: float64

#### [문제 9] [문제 6]에서 생성한 신생아 위험 여부를 예측하는 로지스틱 회귀 모델의 성능을 측정하고자 한다. 분류를 위한 경계값을 0.02로 정의했을 때 분류기의 예측 정밀도(precision)를 구하시오. (10점)
- 테스트 데이터 셋으로 계산하시오.
- 정밀도는 신생아의 위험 여부(at_risk=TRUE)인 것을 기준으로 구하시오.
- 소수점에서 셋째 자리까지만 표시하고 나머진 버리시오.

(답안예시) 0.567

In [21]:
pred = model_fit.predict(test.drop("at_risk", axis = 1))

In [22]:
pred.head()

0    0.008272
3    0.062609
4    0.005382
5    0.010145
9    0.185118
dtype: float64

In [23]:
print(classification_report(test["at_risk"], (pred > 0.02) + 0, digits = 3))

              precision    recall  f1-score   support

           0      0.990     0.799     0.884     11135
           1      0.039     0.511     0.073       180

    accuracy                          0.794     11315
   macro avg      0.515     0.655     0.479     11315
weighted avg      0.975     0.794     0.871     11315



#### [문제 10] [문제 6]에서 생성한 신생아 위험 여부를 예측하는 로지스틱 회귀 모델의 성능 중 AUC-ROC 지표를 구하시오. (10점)
- 테스트 데이터 셋으로 계산하시오.
- 소수점에서 셋째 자리까지만 표시하고 나머진 버리시오.

(답안예시) 0.123

In [25]:
fpr, tpr, thresholds = roc_curve(test["at_risk"], pred)
auc(fpr, tpr)

0.6847156114354138